# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 11:08AM,238867,21,339290060,"NBTY Global, Inc.",Released
1,Jun 6 2022 11:03AM,238866,10,1044,"Amcyte Pharma, Inc.",Released
2,Jun 6 2022 11:02AM,238861,10,MET210878,"Methapharm, Inc.",Released
3,Jun 6 2022 11:02AM,238861,10,MET210879,"Methapharm, Inc.",Released
4,Jun 6 2022 11:02AM,238861,10,MET210880,"Methapharm, Inc.",Released
5,Jun 6 2022 11:02AM,238861,10,MET210881,"Methapharm, Inc.",Released
6,Jun 6 2022 11:02AM,238861,10,MET210886,"Methapharm, Inc.",Released
7,Jun 6 2022 11:02AM,238861,10,MET210887,"Methapharm, Inc.",Released
8,Jun 6 2022 11:02AM,238861,10,MET210888,"Methapharm, Inc.",Released
9,Jun 6 2022 11:02AM,238861,10,MET210890,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,238861,Released,14
38,238864,Released,16
39,238865,Released,7
40,238866,Released,1
41,238867,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238861,NaN,NaN,14.0
238864,NaN,NaN,16.0
238865,NaN,NaN,7.0
238866,NaN,NaN,1.0
238867,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238813,0.0,18.0,10.0
238820,4.0,2.0,8.0
238822,4.0,3.0,7.0
238824,0.0,0.0,58.0
238828,0.0,22.0,25.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238813,0,18,10
238820,4,2,8
238822,4,3,7
238824,0,0,58
238828,0,22,25


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238813,0,18,10
1,238820,4,2,8
2,238822,4,3,7
3,238824,0,0,58
4,238828,0,22,25


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238813,,18,10
1,238820,4,2,8
2,238822,4,3,7
3,238824,,,58
4,238828,,22,25


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 11:08AM,238867,21,"NBTY Global, Inc."
1,Jun 6 2022 11:03AM,238866,10,"Amcyte Pharma, Inc."
2,Jun 6 2022 11:02AM,238861,10,"Methapharm, Inc."
16,Jun 6 2022 11:02AM,238864,10,"Methapharm, Inc."
32,Jun 6 2022 11:02AM,238865,10,"Methapharm, Inc."
39,Jun 6 2022 10:49AM,238860,10,"CLINUVEL, Inc."
40,Jun 6 2022 10:41AM,238858,19,"Innogenix, LLC"
57,Jun 6 2022 10:28AM,238856,19,Eli Lilly and Company
58,Jun 6 2022 10:26AM,238857,20,Sartorius Stedim North America
59,Jun 6 2022 10:17AM,238851,15,"Carwin Pharmaceutical Associates, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 11:08AM,238867,21,"NBTY Global, Inc.",,,1
1,Jun 6 2022 11:03AM,238866,10,"Amcyte Pharma, Inc.",,,1
2,Jun 6 2022 11:02AM,238861,10,"Methapharm, Inc.",,,14
3,Jun 6 2022 11:02AM,238864,10,"Methapharm, Inc.",,,16
4,Jun 6 2022 11:02AM,238865,10,"Methapharm, Inc.",,,7
5,Jun 6 2022 10:49AM,238860,10,"CLINUVEL, Inc.",,,1
6,Jun 6 2022 10:41AM,238858,19,"Innogenix, LLC",,14,3
7,Jun 6 2022 10:28AM,238856,19,Eli Lilly and Company,,,1
8,Jun 6 2022 10:26AM,238857,20,Sartorius Stedim North America,,1,
9,Jun 6 2022 10:17AM,238851,15,"Carwin Pharmaceutical Associates, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 11:08AM,238867,21,"NBTY Global, Inc.",1,,
1,Jun 6 2022 11:03AM,238866,10,"Amcyte Pharma, Inc.",1,,
2,Jun 6 2022 11:02AM,238861,10,"Methapharm, Inc.",14,,
3,Jun 6 2022 11:02AM,238864,10,"Methapharm, Inc.",16,,
4,Jun 6 2022 11:02AM,238865,10,"Methapharm, Inc.",7,,
5,Jun 6 2022 10:49AM,238860,10,"CLINUVEL, Inc.",1,,
6,Jun 6 2022 10:41AM,238858,19,"Innogenix, LLC",3,14,
7,Jun 6 2022 10:28AM,238856,19,Eli Lilly and Company,1,,
8,Jun 6 2022 10:26AM,238857,20,Sartorius Stedim North America,,1,
9,Jun 6 2022 10:17AM,238851,15,"Carwin Pharmaceutical Associates, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 11:08AM,238867,21,"NBTY Global, Inc.",1,,
1,Jun 6 2022 11:03AM,238866,10,"Amcyte Pharma, Inc.",1,,
2,Jun 6 2022 11:02AM,238861,10,"Methapharm, Inc.",14,,
3,Jun 6 2022 11:02AM,238864,10,"Methapharm, Inc.",16,,
4,Jun 6 2022 11:02AM,238865,10,"Methapharm, Inc.",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 11:08AM,238867,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 6 2022 11:03AM,238866,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
2,Jun 6 2022 11:02AM,238861,10,"Methapharm, Inc.",14.0,NaN,NaN
3,Jun 6 2022 11:02AM,238864,10,"Methapharm, Inc.",16.0,NaN,NaN
4,Jun 6 2022 11:02AM,238865,10,"Methapharm, Inc.",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 11:08AM,238867,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 6 2022 11:03AM,238866,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
2,Jun 6 2022 11:02AM,238861,10,"Methapharm, Inc.",14.0,0.0,0.0
3,Jun 6 2022 11:02AM,238864,10,"Methapharm, Inc.",16.0,0.0,0.0
4,Jun 6 2022 11:02AM,238865,10,"Methapharm, Inc.",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3582651,326.0,26.0,4.0
12,477668,25.0,24.0,0.0
15,1433073,47.0,24.0,0.0
19,955368,13.0,16.0,4.0
20,716530,11.0,1.0,0.0
21,238867,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3582651,326.0,26.0,4.0
1,12,477668,25.0,24.0,0.0
2,15,1433073,47.0,24.0,0.0
3,19,955368,13.0,16.0,4.0
4,20,716530,11.0,1.0,0.0
5,21,238867,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,326.0,26.0,4.0
1,12,25.0,24.0,0.0
2,15,47.0,24.0,0.0
3,19,13.0,16.0,4.0
4,20,11.0,1.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,326.0
1,12,Released,25.0
2,15,Released,47.0
3,19,Released,13.0
4,20,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,4.0,0.0,0.0,4.0,0.0,0.0
Executing,26.0,24.0,24.0,16.0,1.0,0.0
Released,326.0,25.0,47.0,13.0,11.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,4.0,0.0,0.0,4.0,0.0,0.0
1,Executing,26.0,24.0,24.0,16.0,1.0,0.0
2,Released,326.0,25.0,47.0,13.0,11.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,4.0,0.0,0.0,4.0,0.0,0.0
1,Executing,26.0,24.0,24.0,16.0,1.0,0.0
2,Released,326.0,25.0,47.0,13.0,11.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()